In [22]:
# Import libraries
import numpy as np
import pandas as pd
import sys
import os
import fitz
from unidecode import unidecode 
import re
import datetime
from spacy import displacy
import spacy
import unicodedata

In [23]:
jt = pd.read_csv("../data/job_titles.csv")

In [24]:
## Functions
def find_role(_words, lvl=0, max_lvl =4):
    it = -1
    flg=False
    for word in _words:
        print('checking if {} in lvl {}'.format(word, lvl))
        it=it+1
        if word in jt['lvl'+str(lvl+1)].dropna().values:
            flg=True
            print('found ', word, 'in ', 'lvl'+str(lvl+1))
            break
    if flg:
        list_to_send = _words[it+1:min(len(_words),it+1+max_lvl-lvl)]
        print('sending the following list to find next level:',list_to_send)
        return ' '.join([find_role(list_to_send, lvl = lvl+1), _words[it]]).strip()
    else:
        return ''   

In [25]:
## Load NLP model (pre-trained)
nlp = spacy.load("en_core_web_md",disable=["ner"])

In [26]:
## Load skills json
skill_pattern_path = "../data/jz_skill_patterns.jsonl"

In [27]:
## Add entity ruler for skills.
## Added before NER to try to get this skills before any other entity predefined
ruler = nlp.add_pipe("entity_ruler", before='ner')
ruler.from_disk(skill_pattern_path)

In [28]:
## Add other customized pattern we wanna find
patterns = [{"label":'EMAIL',"pattern":[{"TEXT":{"REGEX":"([^@|\s]+@[^@]+\.[^@|\s]+)"}}]}]
ruler.add_patterns(patterns)

In [29]:
from nltk.corpus import stopwords
stops = stopwords.words("english")

# Get Sections

In [30]:
cv_folder = '../data/cv'

In [31]:
cv_file_name = 'CV Melvin Leal.pdf'#'Resume-Franklin-Herrera (1).pdf'
#cv_file_name = '202206 Javier Andújar CV.pdf'

In [12]:
DIGITIZED_FILE = os.path.join(cv_folder,cv_file_name)

In [13]:
sections_exps = {'experience':
                 [  
                     'PROFESSIONAL EXPERIENCE','WORK EXPERIENCE','EMPLOYMENT HISTORY','CAREER HISTORY',
                     'EXPERIENCIA PROFESIONAL',
                     'EXPERIENCE', 'Experience','EXPERIENCIA'
                 ],
                 'study':
                 [
                      'EDUCATION','EDUCACION','ESTUDIOS'
                 ],                 
                  'certificates':
                 [
                     'CERTIFICATES', 'CERTIFICATIONS', 'CERTIFICADOS','CERTIFICACIONES'
                 ],                 
                  'trainings':
                 [
                     'TRAININGS', 'COURSES', 'CAPACITACIONES'
                 ],                 
                  'skills':
                 [
                     'SKILLS','HABILIDADES'
                 ]
                 
                 
                }
sections_dict = {"text": [], "line": [], "section":[], 'size':[], 'len':[]}

list_lines = []
counter = 0
with fitz.open(DIGITIZED_FILE) as doc:

    for page in doc: # Iterate all pages in the document

        file_dict = page.get_text('dict', 
                                  sort=True,flags= fitz.TEXT_PRESERVE_WHITESPACE
                                   ) # Get the page dictionary  - Natural order
        block = file_dict['blocks']
        for a in block:   
            if a["type"] == 0:
                for line in a['lines']:
                    flg_collapse=False
                    complete_line = ''.join({j['text'] for j in line['spans']})
                    complete_line = complete_line.replace('  ',' ')
                    
                    if re.search('[A-Z0-9]{1,2} [A-Z0-9]{1,2} [A-Z0-9]{1,2} [A-Z0-9]{1,2}',complete_line, ):
                        print(re.search('[A-Z0-9]{1,2} [A-Z0-9]{1,2} [A-Z0-9]{1,2} [A-Z0-9]{1,2}',complete_line))
                        flg_collapse = True

                    for span in line['spans']:
                        span['text'] =''.join(c for c in unicodedata.normalize('NFD', span['text'])   if unicodedata.category(c) != 'Mn')
                        if flg_collapse:
                            span['text'] = span['text'].replace('  ','__').replace(' ','').replace('__',' ') 
                        list_lines.append(span)
                        ## process each pattern
                        for section,patts in sections_exps.items():
                            for patt in patts:
                                if patt.lower() in span['text'].lower():
                                    sections_dict['text'].append(span['text'])
                                    sections_dict['line'].append(counter)
                                    sections_dict['section'].append(section)
                                    sections_dict['size'].append(span['size'])
                                    sections_dict['len'].append(len(patt.split(' ')))
                        counter=counter+1

In [14]:
sections_dict =pd.DataFrame(sections_dict).sort_values(by=['section', 'size','len', 'line'], ascending=False)
sections_dict

,text,line,section,size,len
4,EDUCATION,192,study,15.96,1
7,SKILLS,317,skills,15.96,1
5,special skills. • Create your own data apps.,249,skills,11.04,1
0,EXPERIENCE,2,experience,15.96,1
1,EXPERIENCE,2,experience,15.96,1
2,Over 10 years of experience,36,experience,11.04,1
3,Over 10 years of experience,36,experience,11.04,1
6,SAP FICO CERTIFICATES,295,certificates,11.04,1


In [15]:
sections_dict.drop_duplicates(['section'],inplace=True, keep='first')
sections_dict.sort_values(by = 'line', inplace=True)
sections_dict['end_section'] = (sections_dict['line'].shift(-1)-1).fillna(len(list_lines)).map(int)
sections_dict

,text,line,section,size,len,end_section
0,EXPERIENCE,2,experience,15.96,1,191
4,EDUCATION,192,study,15.96,1,294
6,SAP FICO CERTIFICATES,295,certificates,11.04,1,316
7,SKILLS,317,skills,15.96,1,342


In [16]:
sections_dict['content'] = sections_dict.apply(lambda x: ' '.join([line['text'] for line in list_lines[x['line']:x['end_section']]]), axis=1)

In [17]:
sections_dict

,text,line,section,size,len,end_section,content
0,EXPERIENCE,2,experience,15.96,1,191,EXPERIENCE Javier Gonzalez FREELANCE D...
4,EDUCATION,192,study,15.96,1,294,EDUCATION BACHELOR'S DEGREE IN SYSTEMS ...
6,SAP FICO CERTIFICATES,295,certificates,11.04,1,316,SAP FICO CERTIFICATES SAP Argentina S.A. — ...
7,SKILLS,317,skills,15.96,1,342,SKILLS Professional ...


In [18]:
work_exp_text = sections_dict[sections_dict['section']=='experience']['content'].values[0]

# Process Work Experience section

In [19]:
def clean_text(txt_content):
    ## Replace -Present variations in text
    txt_content = re.sub(r'[ -]\b(at)?present\b',' '+datetime.date.today().strftime(format = '%Y %m'), txt_content, flags=re.IGNORECASE)
    ## Replace -Present variations in text
    txt_content = re.sub(r'[ -][aA]ctual(idad)?',' '+datetime.date.today().strftime(format = '%Y %m'), txt_content)
    ## Replace , and -
    txt_content = re.sub(r'[,-]'," ", txt_content)
    ## Remove duplicated spaces
    #txt_content = re.sub(r' +'," ", txt_content)
    txt_content = txt_content.replace(' de ', ' ')
    
    months_dict = {
        'enero':'January',
        'febrero':'February',
        'marzo':'March',
        'abril':'April',
        'mayo':'May',
        'junio':'June',
        'julio':'July',
        'agosto':'August',
        'septiembre':'September',
        'setiembre':'September',
        'octubre':'October',
        'noviembre':'November',
        'diciembre':'December',
        
    }
    for month,val in months_dict.items():
        txt_content = txt_content.lower().replace(month,val, )
        
    ## Remove accents
    txt_content =''.join(c for c in unicodedata.normalize('NFD', txt_content)   if unicodedata.category(c) != 'Mn')
    return txt_content

In [20]:
text = clean_text(work_exp_text)

In [ ]:
work_exp_text

In [21]:
print(text)

experience  javier gonzalez        freelance data developer  andujar    buenos aires city — feb 2021 2023 03      sr. data enginner    tasks performed:    • sql developer • etl / etd processes • reporting    technologies and tools:             • sql server on azure • power bi • python (pandas  bs  vader)      database developer  liszt soft srl    buenos aires city — nov 2018 jan 2021        tasks performed:  over 10 years of experience    • sql developer • creation of etl / etd processes • reporting  working in database    technologies and tools:   development and analysis.      • sql server 2016 in azure • sql server reporting services (ssrs) •  power bi • power query        sql etl developer / dba  recovery group    buenos aires city — mar 2016 oct 2018              analysis development and maintenance of datawarehouse for the steel  industry. in charge of the operational continuity of the passive cycle  dwh.      tasks performed:      • sql developer. • database administrator. • con

In [ ]:
## Convert to nlp doc and remove special characters
doc = nlp(text)

lemmatized = list()
for word in doc:
    lemma = word.lemma_.strip()
    lemmatized.append(lemma)
    """
    if lemma:
        if lemma not in stops:
            lemmatized.append(lemma)
    """
text2 = " ".join(lemmatized)

text2 = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",text2)


doc2 = nlp(text2)
print(text2)

In [ ]:
doc2

In [ ]:
for j in doc2.ents:
    print(j, j.label_)

In [ ]:
# Identify different dates format in the text

## Define patterns 
exps = [
    r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.-/])\D?)?,?(\s)?((19[7-9]\d|20\d{2})|\d{2})',
    r'\b(?:Ene(?:ro)?|Feb(?:rero)?|Mar(?:zo)?|Abr(?:il)?|May(?:o)?|Jun(?:io)?|Jul(?:io)?|Ago(?:sto)?|Sep(?:tiembre)?|Set(?:iembre)?|Oct(?:ubre)?|(Nov|Dic)(?:iembre)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.-/])\D?)?,?(\s)?((19[7-9]\d|20\d{2})|\d{2})',
    
    r'\b(\d{2} )?(\d{2} )(20\d{2})\b(?!\d)',
    r'\b(\d{2}\/)?(\d{2}\/)(20\d{2})\b(?!\d)',
    r'\b(\d{2}\.)?(\d{2}\.)(20\d{2})\b(?!\d)',
    r'\b(\d{2})?(\d{2})(20\d{2})\b(?!\d)',
    
    r'\b(20\d{2} )(\d{2})( \d{2})?\b',
    r'\b(20\d{2}\/)(\d{2})(\/\d{2})?\b',
    r'\b(20\d{2}\.)(\d{2})(\.\d{2})\b'
]
bag_dates = [] ## will store dates

## Collect useful data from date text
dates_dict = {
    'span_start':[],
    'span_end': [], 
    'span_text':[], 
    'date_formated':[]
}

## process each pattern
for reg in exps:
    for match in re.finditer(reg, doc2.text, flags=re.IGNORECASE): ## if pattern match
        start, end = match.span()
        span = doc2.char_span(start, end)
        # This is a Span object or None if match doesn't map to valid token sequence
        if span is not None:
            bag_dates.append([span,start,end,pd.to_datetime(span.text)]) ## add matched date string
            ## Save data from date matched
            print("Found match:", span.text)
            dates_dict['span_start'].append(start)
            dates_dict['span_end'].append(end)
            dates_dict['span_text'].append(span.text)
            dates_dict['date_formated'].append(pd.to_datetime(span.text))

In [ ]:
# Get work history dates
## Get pairs of dates and decide if they are work history dates
dates_df = (
    pd.DataFrame(dates_dict)
    .sort_values(by ='span_start', ascending=True)
    .drop_duplicates(['span_start','span_end']))

dates_df['close_to_previous'] = ((dates_df['span_start'] - dates_df['span_end'].shift(1))<10) *1 ## pair of history dates should be close

dates_df.loc[(dates_df['close_to_previous']==1 )
             & (dates_df['date_formated'] > dates_df['date_formated'].shift(1))
             ,'type'] = 'end'
dates_df.loc[(dates_df['type'].shift(-1)=='end') & (dates_df['date_formated'] < dates_df['date_formated'].shift(-1))
             ,'type'] = 'start'


dates_df = dates_df[dates_df['type'].notnull()] ## filter only valid pairs

## Pivot pairs
dates_df['pair'] = 1
dates_df['pair']  = np.ceil(dates_df['pair'].cumsum()/2)
dates_df = dates_df.pivot(index = 'pair'
                          , columns = 'type'
                          , values = ['date_formated','span_start','span_end'])

## Calculate experience lenght in months
dates_df['duration'] = (dates_df['date_formated']['end'] - dates_df['date_formated']['start']).dt.total_seconds()/3600/24//30

dates_df.sample(min(5,len(dates_df)))

In [ ]:
dates_df.columns = ['_'.join([str(x),str(y)]) for x,y in dates_df.columns]

In [ ]:
dates_df

In [ ]:
# Calculate total experience
a = dates_df[['date_formated_start','date_formated_end']]
total_months = pd.DataFrame()
for _,i in a.iterrows():
    total_months = pd.concat([
        total_months,
        pd.DataFrame(pd.date_range(i['date_formated_start'],i['date_formated_end'] ,freq="MS" ,inclusive='both' ))
    ], axis = 0, ignore_index=True)
total_months.drop_duplicates(inplace=True)
print('First and Last months of experience:', total_months[0].min(), ' - ', total_months[0].max())
print('Total months of experience: {}'.format(total_months.shape[0]+1))

In [ ]:
dates_df['role_text'] = dates_df.apply(lambda x: doc2.text[max(0,x['span_start_start']-80):min(len(doc2.text), x['span_end_end']+80)].split(' ')[::-1], axis=1)

In [ ]:
dates_df['role_text'] = dates_df['role_text'].map(lambda x: [i for i in x if i!=''])

In [ ]:
dates_df['role_name'] = dates_df['role_text'].map(find_role, )

In [ ]:
dates_df

In [ ]:
# Get Dataframe from entities found by NLP
## With entity ruler or NER
labels  = pd.DataFrame(columns = ['label','span_start','span_end','text'])
for e in doc2.ents:
    row = {}
    row['label'] = [e.label_]
    row['span_start'] = [e.start_char]
    row['span_end'] = [e.end_char]
    row['text'] = [e.text]
    labels = pd.concat([labels,pd.DataFrame(row)] , ignore_index=False)
labels.sample(min(5, len(labels)))

In [ ]:
def clean_skills(txt_content):
    txt_content = txt_content.str.lower().str.replace(' ','_').str.replace('-','_')
    return txt_content

In [ ]:
# Match skills with work history

## Format work history dates
df_date_to_concat = (
    dates_df[['span_start_start','span_end_end','date_formated_start','date_formated_end','duration_']]
    .rename(columns = {'span_start_start':'span_start', 'span_end_end':'span_end','duration_':'duration'})
)

## Add entity label
df_date_to_concat['label']='DATE'

## Place skills within corresponding work history dates
df_concat = (
    pd.concat([df_date_to_concat,labels],
              axis=0,
              ignore_index=False)
    .sort_values(by = 'span_start')
)

## Fill same work history skills
df_concat['date_formated_start'] = df_concat['date_formated_start'].ffill()
df_concat['date_formated_end'] = df_concat['date_formated_end'].ffill()
df_concat['duration'] = df_concat['duration'].ffill()

## Remove duplicates in case same skill found more than once in the work history
df_concat.drop_duplicates(['date_formated_start','text'], inplace=True)
df_concat.sample(5)

In [ ]:
## Keep only skills and summary to get total experience by skill
df_concat = (
    df_concat[df_concat['label']=='SKILL']
    .groupby('text')
    .agg({'duration':'sum','date_formated_start':'min','date_formated_end':'max', 'label':'count' })
    .reset_index()
)

df_concat['years'] = (df_concat['duration']/12).round(1) ## Calculate years

## Get thresholds to match with scores table later
df_concat['years_threshold'] = pd.cut(
    df_concat['years'],
    bins = [0,0.5,1,2,3,4,100],
    labels = [0,0.5,1,2,3,4],
    right=False)
df_concat.sample(5)

In [ ]:
## Clean skills in main dataser
df_concat['skill'] = clean_skills(df_concat['text'])

In [ ]:
## Save data
data_to_save = df_concat[['skill','label','date_formated_start','date_formated_end','duration']]
data_to_save.columns = ['skill','counter','start_date', 'end_date','months_duration']
data_to_save['applicant_name'] = "Jose Gutierrez"
display(data_to_save.sample(5))
data_to_save.to_csv('work_history_sample.csv', index=False, encoding='utf8', mode='a', header=False)

# Calculate Profile Score

In [ ]:
# Add Scores
## It contains the association between months of experience -> score
df_scores = pd.read_excel('../data/config.xlsx', sheet_name = 'scores')
df_concat = df_concat.merge(df_scores, how = 'left', on = 'years_threshold' , validate = 'many_to_one')
del df_scores
df_concat.sample(5)

In [ ]:
## Add skill - Expertise
df_sk_ex = pd.read_excel('../data/config.xlsx', sheet_name = 'expertise_skill')
df_sk_ex['skill'] = clean_skills(df_sk_ex['skill'])
df_concat = df_concat.merge(df_sk_ex, how = 'left', on = 'skill' , validate = 'many_to_one')
df_concat.sample(5)

In [ ]:
## Add skill - role
df_sk_role = pd.read_excel('../data/config.xlsx', sheet_name = 'skills_role')
df_sk_role['skill'] = clean_skills(df_sk_role['skill'])
df_concat = df_concat.merge(df_sk_role, how = 'left', on = 'skill' )
df_concat.sample(5)

In [ ]:
## Add skill - Expertise
df_ex_role = pd.read_excel('../data/config.xlsx', sheet_name = 'expertise_role')
df_concat = df_concat.merge(df_ex_role.rename(columns = {'weight':'weight_expertise'}), how = 'left', on = ['expertise_area','role' ])
df_concat.sample(5)

In [ ]:
#Calculate final score
df_concat['final_score'] = df_concat['Score'] * df_concat['weight'] *df_concat['weight_expertise'] 

In [ ]:
df_concat

In [ ]:
## Summaryze
df_concat.groupby(['role']).agg({'final_score':'sum'})

In [ ]:
df_concat.groupby(['role','expertise_area']).agg({'final_score':'sum'})

In [ ]:
df_concat[['skill','years','Score']].drop_duplicates()

In [ ]:
## Define data to save
## Skills w months
## contact info
# skills counter (for word cloud)
## Try to capture role

In [ ]:
## Save data

In [38]:
os.path.join('../data/output', 'lala')

'../data/output/lala'

In [34]:
left = pd.DataFrame({'a': [1, 5, 10,4,3], 'left_val': ['a', 'b', 'c','e','f']})

In [35]:
right = pd.DataFrame({'a': [1, 2, 3, 6, 7],'right_val': [1, 2, 3, 6, 7]})

In [36]:
pd.merge_asof(left, right, on='a', )

ValueError: left keys must be sorted

In [4]:
lst = ['Abogado',
'Abogado de inmigracion',
'Abogado de patentes',
'Abogado por contrato',
'Actor',
'Administrador',
'Administrador de base de datos',
'Administrador de casos',
'Administrador de eventos',
'Administrador de hogares de ancianos',
'Administrador de la propiedad',
'Administrador de Linux',
'Administrador de portafolio',
'Administrador de red',
'Administrador de seguridad',
'Administrador de sistema',
'Administrador de sistemas',
'Administrador regional',
'Agente de aduana',
'Agente de arrendamiento',
'Agente de bienes raices',
'Agente De Compras',
'Agente de firma',
'Agente de libertad condicional',
'Agente de reservas',
'Agente de seguros',
'Agente de viajes',
'Agente deportivo',
'Agente especial',
'Agricultor',
'Agronegocio',
'Agronomo',
'Ajustador automatico',
'Ajustador de reclamos',
'Ajustador de seguros',
'Alguacil',
'Alumno',
'Ama de casa',
'Analista',
'Analista actuarial',
'Analista ambiental',
'Analista cuantitativo',
'Analista de aplicaciones',
'Analista de atencion medica',
'Analista de base de datos',
'Analista de bienes raices',
'Analista de Cadena de Suministro',
'analista de calidad',
'Analista de comportamiento',
'Analista de control de calidad',
'Analista de credito',
'Analista de datos',
'Analista de energia',
'Analista de fraude',
'Analista de inteligencia',
'Analista de inteligencia de negocios',
'Analista de investigacion',
'Analista de Investigacion de Mercado',
'Analista de investigacion de renta variable',
'Analista de marketing',
'Analista de negocios',
'Analista de operaciones',
'Analista de politicas',
'Analista de precios',
'Analista de presupuestos',
'Analista de productos',
'Analista de proyectos',
'Analista de recursos humanos',
'Analista de redes',
'Analista de renta variable',
'Analista de riesgos',
'Analista de seguridad',
'Analista de seguridad cibernetica',
'Analista de seguridad de la informacion',
'Analista de servicio al cliente',
'Analista de SIG',
'Analista de sistemas',
'Analista financiero',
'Analizador de sistemas',
'Anestesiologo',
'Aparejador',
'Aprendiz de electricidad',
'Aprendiz de electricista',
'Archivero',
'Arquitecto',
'Arquitecto de datos',
'Arquitecto de red',
'Arquitecto de seguridad',
'Arquitecto de software',
'Arquitecto de Soluciones',
'Arquitecto Empresarial',
'Artista',
'Artista de produccion',
'Artista grafico',
'Asegurador',
'Asesor de servicio',
'Asesor financiero',
'Asesor residente',
'Asistente',
'Asistente administrativo ejecutivo',
'Asistente administrativo medico',
'Asistente clinico',
'Asistente comprador',
'Asistente de actividades',
'Asistente de administrador de la propiedad',
'Asistente de arte',
'Asistente de atencion al paciente',
'Asistente de Compras',
'Asistente de consultorio medico',
'Asistente de contabilidad',
'Asistente de cuidado personal',
'Asistente de enseñanza',
'Asistente de fisioterapeuta',
'Asistente de fisioterapia',
'Asistente de instruccion',
'Asistente de investigacion',
'Asistente de laboratorio',
'Asistente de maestra',
'Asistente de maestro',
'Asistente de Marketing',
'Asistente de nutricion',
'Asistente de oficina',
'Asistente de ortodoncia',
'Asistente de patologia',
'Asistente de produccion',
'Asistente de programa',
'Asistente de Proyecto',
'Asistente de radiologia',
'asistente de recursos humanos',
'Asistente de salud',
'Asistente de servicios sociales',
'Asistente de suscripcion',
'Asistente de terapia ocupacional',
'Asistente de terapia ocupacional certificada',
'Asistente del gerente de la tienda',
'Asistente dietetico',
'Asistente editorial',
'Asistente Ejecutiva',
'Asistente escolar',
'Asistente graduado',
'Asistente inmobiliario',
'Asistente legal',
'Asistente legislativo',
'Asistente medico',
'Asistente medico certificado',
'Asistente personal',
'Asistente principal',
'Asistente quirurgico',
'Asistente Residente',
'Asistente social',
'Asistente veterinario',
'Asociado de almacen',
'Asociado de investigacion clinica',
'Asociado de Marketing',
'Asociado de ventas',
'Asociado Ventas al por menor',
'Auditor',
'Auxiliar de contabilidad',
'Auxiliar de enfermeria',
'Auxiliar de vuelo',
'Ayudante Administrativo',
'Ayudante de camara',
'Ayudante de dentista',
'Ayudante de instruccion',
'Ayudante de lavanderia',
'Ayudante del conductor',
'Ayudante electrico',
'Ayudante legal',
'Ayudante psiquiatrico',
'Banquero',
'Banquero personal',
'Banquete',
'Barbero',
'Barista',
'Barman',
'bibliotecario',
'Biller medico',
'Bioestadistico',
'Biologo',
'Cajero',
'Camarera',
'Campo Tecnico de Servicio',
'Capellan',
'Cardiologo',
'Carpintero',
'Cartero',
'CEO',
'Cerrajero',
'Chef ejecutivo',
'Chofer',
'Cientifico',
'Cientifico ambiental',
'Cientifico biomedico',
'Cientifico de comida',
'Cientifico de datos',
'Cientifico de la computacion',
'Cientifico de laboratorio clinico',
'Cientifico de Laboratorio Medico',
'Cientifico de materiales',
'Cientifico forense',
'Cientifico Medico',
'Cientifico social',
'CIO',
'CNA',
'Cobrador',
'Cocina de linea',
'Cocinero',
'Codificador medico',
'Codificador medico remoto',
'Combatiente',
'Comerciante',
'Comerciante minorista',
'Comerciante visual',
'Compositor',
'Comprador',
'Conductor',
'Conductor de autobus',
'Conductor de camion',
'Conductor de furgoneta',
'Conductor de taxi',
'Conductor del autobus escolar',
'Conductor del montacargas',
'Consejero',
'Consejero de abuso de sustancias',
'Consejero de admisiones',
'Consejero de la escuela',
'Consejero de salud mental',
'Consejero estudiantil',
'Consejero financiero',
'Consejero general',
'Consejero profesional con licencia',
'Consejero residencial',
'Conserje',
'Consultor',
'Consultor ambiental',
'Consultor de arrendamiento',
'Consultor de atencion medica',
'Consultor de enfermeria',
'Consultor de ingenieria',
'Consultor de marketing',
'Consultor de negocios',
'Consultor de nutricion',
'Consultor de seguridad',
'Consultor de ventas',
'Consultor educacional',
'Contable',
'Contador',
'Contador de impuestos',
'Contador de personal',
'Contratista independiente',
'Controlador',
'Controlador asistente',
'Controlador CDL',
'Controlador de calidad',
'Controlador de trafico aereo',
'COO',
'Coordinador',
'Coordinador Administrativo',
'Coordinador de actividades',
'Coordinador de admision',
'Coordinador de atencion al paciente',
'Coordinador de bienestar',
'Coordinador de bodas',
'Coordinador de Cuenta',
'Coordinador de educacion',
'coordinador de entrenamientos',
'Coordinador de eventos',
'Coordinador de investigacion',
'Coordinador de la unidad de salud',
'Coordinador de Logistica',
'Coordinador de marketing',
'Coordinador de oficina',
'Coordinador de produccion',
'Coordinador de programa',
'Coordinador de reclutamiento',
'Coordinador de Recursos Humanos',
'Coordinador de redes sociales',
'Coordinador de Seguridad',
'Coordinador del proyecto',
'Coordinador voluntario',
'Corrector de pruebas',
'Corredor',
'Corredor de alimentos',
'Corredor de bienes raices',
'Cortador de carne',
'Creador de herramientas',
'Creador de prestamos',
'Cro',
'CTO',
'Cuidador',
'Cuidador del paciente',
'Dentista',
'Desarrollador',
'Desarrollador de Android',
'Desarrollador de aplicaciones',
'Desarrollador de C ++',
'Desarrollador de C#',
'Desarrollador de ETL',
'Desarrollador de informes',
'Desarrollador de Java',
'Desarrollador de JavaScript',
'Desarrollador de juegos',
'Desarrollador de la unidad',
'Desarrollador de pila completa',
'Desarrollador de Python',
'Desarrollador de Salesforce',
'Desarrollador de software',
'Desarrollador de SQL',
'Desarrollador de UI',
'Desarrollador de WordPress',
'Desarrollador frontal',
'Desarrollador HTML',
'Desarrollador junior',
'Desarrollador movil',
'Desarrollador PHP',
'Desarrollador remoto',
'Desarrollador web',
'Descifrador',
'Despachador',
'detective',
'Dibujante',
'Director',
'Director creativo',
'Director de actividades',
'Director de arte',
'Director de atencion',
'Director de campamento',
'Director de comunicaciones',
'Director de enfermeria',
'Director de escena',
'director de Finanzas',
'Director de Formacion',
'Director de Ingenieria',
'Director de Marketing',
'Director de operaciones',
'Director de programa',
'director de Recursos Humanos',
'Director de seguridad',
'Director de Servicio de Alimentos',
'Director de TI',
'Director del programa',
'Director ejecutivo',
'Director medico',
'Director tecnico',
'Diseñador',
'Diseñador arquitectonico',
'Diseñador de interfaz de usuario',
'Diseñador de interiores',
'Diseñador de juegos',
'Diseñador de sonido',
'Diseñador de tuberias',
'Diseñador de UX',
'Diseñador de videojuegos',
'Diseñador grafico',
'Diseñador industrial',
'Diseñador instruccional',
'Diseñador junior',
'Diseñador tecnico',
'Diseñador visual',
'Diseñador web',
'Diseño textil',
'DJ',
'Doctor',
'Dueño',
'Dueño del producto',
'Editor',
'Editor de contenido',
'Editor de video',
'Educador',
'Educador de enfermeria',
'Educador de salud',
'Ejecutivo',
'Ejecutivo de cuentas',
'Ejecutivo de ventas',
'Electricista',
'Electricista industrial',
'Embajador',
'Embajador de la marca',
'Empleado',
'Empleado de almacen',
'Empleado de entrada de datos',
'Empleado de envio',
'Empleado de farmacia',
'Empleado de la nomina',
'Empleado de la unidad',
'Empleado de oficina',
'Empleado de registros medicos',
'Empleado legal',
'Encargado del edificio',
'Enfermera anestesista',
'Enfermera de cuidado de heridas',
'Enfermera de dialisis',
'Enfermera de hospicio',
'Enfermera de infusion',
'Enfermera de investigacion',
'Enfermera de la escuela',
'Enfermera de oficina',
'Enfermera de parto y parto',
'Enfermera de revision de utilizacion',
'Enfermera de salud',
'Enfermera de salud ocupacional',
'Enfermera graduada',
'Enfermero',
'Ensamblador',
'Ensayador',
'Entrenador',
'Entrenador atletico',
'Entrenador corporativo',
'Entrenador de fuerza y ​​acondicionamiento',
'Entrenador de salud',
'Entrenador personal',
'Entrenador vital',
'Envasador',
'Epidemiologo',
'Escritor',
'Escritor de contenido',
'Escritor de propuestas',
'Escritor independiente',
'Escritor medico',
'Escritor tecnico',
'Especialista',
'especialista ambiental',
'Especialista clinico',
'Especialista en comportamiento',
'Especialista en comunicaciones',
'Especialista en contrato',
'Especialista en documentacion clinica',
'Especialista en educacion',
'Especialista en entrenamiento',
'Especialista en facturacion',
'Especialista en fitness',
'Especialista en inventario',
'Especialista en la vida infantil',
'Especialista en marketing',
'Especialista en medios',
'Especialista en Recursos Humanos',
'especialista en salarios',
'Especialista en Social Media',
'Especialista en soporte entre pares',
'Especialista en TI',
'Estadistico',
'Esteticista',
'Estilista',
'Estimador de construccion',
'Estimador de costos',
'Estratega',
'Estudiante de enfermeria',
'Examinador',
'Expedidor',
'Facilitador',
'Facultad de enfermeria',
'Farmaceutico',
'Fiscal de distrito',
'Fisico',
'Fisioterapeuta',
'Flebotomista',
'Fotografo',
'Funcionario de prisiones',
'Geologo',
'Gerente',
'Gerente central',
'Gerente comunitario',
'Gerente de apartamentos',
'Gerente de banco',
'Gerente de Calidad',
'Gerente de compras',
'Gerente de comunicaciones',
'Gerente de construccion',
'Gerente de cuentas',
'Gerente de cuidado',
'Gerente de distrito',
'Gerente de enfermera',
'Gerente de finanzas',
'Gerente de granja',
'Gerente de la casa',
'Gerente de la flota',
'Gerente de la instalacion',
'Gerente de la planta',
'Gerente de la tienda',
'Gerente de las instalaciones',
'Gerente de Logistica',
'Gerente de marca',
'Gerente de Medio Ambiente',
'Gerente de oficina',
'Gerente de Operaciones',
'Gerente de piezas',
'Gerente de practica',
'Gerente de prevencion de perdidas',
'Gerente de producto',
'Gerente de proyecto',
'Gerente de rama',
'Gerente de Recursos Humanos',
'Gerente de redes sociales',
'Gerente de relaciones',
'Gerente de seguridad',
'Gerente de Servicio Automotriz',
'Gerente de Servicio de Alimentos',
'Gerente de TI',
'Gerente de transporte',
'Gerente de ventas',
'Gerente de viaje',
'Gerente del bar',
'Gerente del restaurante',
'Gerente del servicio al cliente',
'Gerente financiero',
'Gerente general',
'Gerente minorista',
'gerente territorial',
'Gestion de productos',
'Gestor de contenidos',
'Guardavia',
'Guardia de seguridad',
'Guardia Nacional del Ejercito',
'Guionista',
'Higienista dental',
'HR Generalista',
'Ingeniero',
'Ingeniero aeroespacial',
'Ingeniero biomedico',
'Ingeniero civil',
'Ingeniero de audio',
'Ingeniero de automatizacion',
'Ingeniero de big data',
'Ingeniero de calidad',
'Ingeniero de campo',
'Ingeniero de construccion',
'Ingeniero de control de calidad',
'Ingeniero de controles',
'Ingeniero de datos',
'Ingeniero de DevOps',
'Ingeniero de diseño',
'Ingeniero de embalaje',
'Ingeniero de Manufactura',
'Ingeniero de materiales',
'Ingeniero de Procesos',
'Ingeniero de Producto',
'Ingeniero de proyectos',
'Ingeniero de pruebas',
'Ingeniero de redes',
'Ingeniero de robotica',
'Ingeniero de seguridad',
'Ingeniero de seguridad cibernetica',
'ingeniero de Servicio de campo',
'Ingeniero de sistemas',
'Ingeniero de software',
'Ingeniero de software junior',
'Ingeniero de soldadura',
'Ingeniero de sonido',
'Ingeniero de validacion',
'Ingeniero de Ventas',
'Ingeniero electrico',
'Ingeniero electronico',
'Ingeniero en hardware',
'Ingeniero estructural',
'Ingeniero industrial',
'Ingeniero jefe',
'Ingeniero Junior',
'Ingeniero Maritimo',
'Ingeniero mecanico',
'Ingeniero nuclear',
'Ingeniero petrolero',
'Ingeniero quimico',
'Inspector',
'Inspector de alimentos',
'Inspector de calidad',
'Inspector de campo',
'Inspector de construccion',
'Inspector de construcciones',
'Inspector de incendios',
'Inspector de la casa',
'Inspector de seguridad',
'Inspector de soldadura certificado',
'Inspector electrico',
'Instalador del gabinete',
'Instructor',
'Instructor adjunto',
'Instructor automotriz',
'Instructor de cosmetologia',
'Instructor de enfermeria',
'Instructor de fitness',
'Instructor de fitness grupal',
'Instructor de justicia penal',
'Instructor de manejo',
'Interprete',
'Inventor',
'Investigador asociado',
'Investigador cientifico',
'Investigador clinico',
'Investigador de antecedentes',
'Investigador de fraude',
'Investigador de incendios',
'Investigador privado',
'Jardinero',
'Jefe de almacen',
'Jefe de bomberos',
'Jefe de equipo',
'Jefe de estado mayor',
'Jefe de policia',
'Jefe de policia',
'Jefe de produccion',
'Lider de enfermera clinica',
'Lingüista',
'Localizador de servicios publicos',
'Locutor de radio',
'Logistico',
'Maestra de primaria',
'Maestro',
'Maestro de español',
'Maestro de scrum',
'Maestro suplente',
'Manejo de paquetes',
'Manipulador de materiales',
'Mantenimiento Electricista',
'Maquinista',
'Masajista',
'Mayordomo',
'Mecanico',
'Mecanico de aviones',
'Mecanico de helicoptero',
'Mecanico diesel',
'Mecanografo',
'Mediador',
'Medical Science Liaison',
'mensajero',
'Mentor',
'Mesero',
'metodista',
'Mezclador',
'Microbiologo',
'Modelo',
'Molinero',
'Musico',
'Navegador de pacientes',
'Negociador',
'Negocio deportivo',
'Niñera',
'Notario publico',
'Nutricionista',
'Obrero',
'Oficial de correcciones',
'Oficial de cumplimiento',
'Oficial de detencion',
'Oficial de la corte',
'Oficial de policia',
'Oficial de prestamos',
'Oficial de prestamos hipotecarios',
'Oficial de seguridad',
'Oficial de Seguridad Publica',
'Oficial del ejercito',
'Oficial militar',
'Oficinista',
'Operador',
'Operador de aguas residuales',
'Operador de CNC',
'Operador de equipo',
'Operador de grua',
'Operador de la computadora',
'Operador de maquinaria',
'Operador de montacargas',
'Operador de plantas',
'Operador de procesos',
'Operador de produccion',
'Operador quimico',
'Organizador',
'Organizador de eventos',
'Orientador',
'Originador de prestamos hipotecarios',
'Panadero',
'Paramedico',
'Paraprofesional',
'Patologo del habla y lenguaje',
'Pediatra',
'Periodista',
'Personal de mantenimiento',
'Piloto',
'Pintor',
'Planeador financiero',
'Planificador',
'Planificador de medios',
'Planificador de transporte',
'Plomero',
'Poeta',
'Portero',
'Practicante',
'Practicante de enfermeria',
'Preparador de impuestos',
'Probador de control de calidad',
'Probador de software',
'Procesador de muestras',
'Procesador de prestamos',
'Procesador hipotecario',
'Produccion de medios',
'Productor',
'Productor de videos',
'Profesional de apoyo directo',
'Profesor',
'Profesor auxiliar',
'Profesor de ciencias',
'Profesor de contabilidad',
'Profesor de educacion fisica',
'Profesor de estudios sociales',
'Profesor de ingles',
'Profesor de instituto',
'Profesor de lectura',
'Profesor de preescolar',
'Profesor frances',
'Profesor Ingles',
'Programador',
'Programador Analista',
'Programador de Java',
'Programador SAS',
'Promotor',
'Psicologo',
'Psiquiatra',
'Quimico',
'Quimico analitico',
'Radiografo',
'Radiologo',
'Recepcionista',
'Recepcionista medico',
'Reclutador',
'Reclutador tecnico',
'Redactor',
'Redocratado',
'Registro empleado',
'Reportero',
'Reportero de la corte',
'Reportero de noticias',
'Reportero deportivo',
'Repostero',
'Representante central',
'Representante de acceso al paciente',
'Representante de Servicio al Cliente',
'Representante de ventas',
'Representante de ventas farmaceuticas',
'Representante de ventas medicas',
'Representante del servicio del paciente',
'Secretaria legal',
'Secretario',
'Secretario de la unidad',
'Secretario medico',
'Selector de pedidos',
'Servidor de banquetes',
'Soldador',
'Subdirector',
'Subgerente',
'Superintendente',
'Superintendente asistente',
'superintendente de construccion',
'Supervisor',
'Supervisor de almacen',
'Supervisor de Labores Domesticas',
'supervisor de mantenimiento',
'Supervisor de produccion',
'Suscriptor hipotecario',
'Tasador',
'Tasador de bienes raices',
'Tecnico',
'Tecnico ambiental',
'Tecnico automotriz',
'Tecnico biologico',
'Tecnico de atencion al paciente',
'Tecnico de audio',
'Tecnico de audio visual',
'Tecnico de avionica',
'Tecnico de ayuda informatica',
'Tecnico de biblioteca',
'Tecnico de cable',
'Tecnico de CAD',
'Tecnico de Calidad',
'Tecnico de campo',
'Tecnico de centro de datos',
'Tecnico de dialisis',
'Tecnico de emergencias medicas',
'Tecnico de equipos biomedicos',
'Tecnico de fisioterapia',
'Tecnico de flebotomia',
'Tecnico de histologia',
'Tecnico de Ingenieria',
'Tecnico de ingeniero',
'Tecnico de instrumentos',
'Tecnico de investigacion',
'Tecnico de la informacion de salud',
'Tecnico de la sala de emergencias',
'Tecnico de laboratorio',
'Tecnico de Mantenimiento',
'Tecnico de monitor',
'Tecnico de NOC',
'Tecnico de PC',
'Tecnico de PLC',
'Tecnico de procesamiento esteril',
'Tecnico de proceso',
'Tecnico de radiologia',
'Tecnico de red',
'Tecnico de registros medicos',
'Tecnico de rehabilitacion',
'Tecnico de salud conductual',
'Tecnico de salud mental',
'Tecnico de servicio',
'Tecnico de soporte de escritorio',
'Tecnico de telecomunicaciones',
'Tecnico de torre',
'Tecnico de turbina eolica',
'Tecnico de ultrasonido',
'Tecnico del departamento de emergencias',
'Tecnico diesel',
'Tecnico electromecanico',
'Tecnico electronico',
'Tecnico en computacion',
'Tecnico en electronica',
'Tecnico en farmacologia',
'Tecnico forestal',
'Tecnico lubricante',
'Tecnico medico',
'Tecnico psiquiatrico',
'Tecnico quimico',
'Tecnico quirurgico',
'Tecnico veterinario',
'Tecnologo cardiovascular',
'Tecnologo de comida',
'Tecnologo de medicina nuclear',
'Tecnologo de resonancia magnetica',
'Tecnologo medico',
'Tecnologo quirurgico',
'Telefonista',
'Telemetria',
'Terapeuta',
'Terapeuta de salud mental',
'Terapeuta matrimonial y familiar',
'Terapeuta ocupacional',
'Terapeuta recreativa',
'Terapeuta respiratorio',
'TI Manager de proyecto',
'Topografo',
'Trabajador de almacen',
'Trabajador de cuidado',
'Trabajador de la construccion',
'Trabajador de salud comunitaria',
'Trabajador de salud mental',
'Trabajador general',
'Trabajador portuario',
'Trabajador social',
'Trabajador social clinico',
'Trabajador social de hospicio',
'Trabajador social medico',
'Traductor',
'Traductor chino',
'Transcriptor',
'Transcriptor medico',
'Transportista de paciente',
'Trepador de torre',
'Tripulante de cabina',
'Tutor',
'Tutor academico',
'Tutor de ingles',
'Tutor frances',
'Vendedor',
'Vendedor telefonico',
'Veteranos',
'Veterinario',
'vicepresidente',
'Vidriero',
'Vigilante',
'Voluntario']

In [ ]:
ruler.patterns[-10:]

In [ ]:
ruler.patterns[1:10]

In [ ]:
print("""{"LOWER":\"""")

In [5]:
a = ["{\"label\":\"JOB_TITLE\",\"pattern\":[" + ','.join(["{\"LOWER\":\""+j.lower().strip()+"\"}" for j in i.split(" ")]) for i in lst]

In [6]:
for i in a:
    print(i + "]}")

{"label":"JOB_TITLE","pattern":[{"LOWER":"abogado"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"abogado"},{"LOWER":"de"},{"LOWER":"inmigracion"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"abogado"},{"LOWER":"de"},{"LOWER":"patentes"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"abogado"},{"LOWER":"por"},{"LOWER":"contrato"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"actor"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"administrador"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"administrador"},{"LOWER":"de"},{"LOWER":"base"},{"LOWER":"de"},{"LOWER":"datos"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"administrador"},{"LOWER":"de"},{"LOWER":"casos"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"administrador"},{"LOWER":"de"},{"LOWER":"eventos"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"administrador"},{"LOWER":"de"},{"LOWER":"hogares"},{"LOWER":"de"},{"LOWER":"ancianos"}]}
{"label":"JOB_TITLE","pattern":[{"LOWER":"administrador"},{"LOWER":"de"},{"LOWER":"la"},{"LOWER":"propiedad"}]}
{"la

In [ ]:
a = [i[-1:0]]